# Trace Introduction
Trace sniffing is supported on two platforms. Setup depends on the platform, so select your platform correctly below:

In [ ]:
TRACE_PLATFORM = 'CW610' # AKA PhyWhisperer
#TRACE_PLATFORM = 'CW305' # CW305 FPGA target board

The CW610 option requires a CW610 PhyWhisperer and a CW308 multi-target board with the MK82F target mounted and programmed with the simpleserial-trace example firmware.

In [ ]:
from chipwhisperer.capture.trace.TraceWhisperer import TraceWhisperer

In [ ]:
if TRACE_PLATFORM == 'CW610':
    SCOPETYPE = 'OPENADC'
    PLATFORM = 'CWLITEARM'
    %run "Helper_Scripts/Setup_Generic.ipynb"
    trace = TraceWhisperer(target)
    # on this platform, minimum trace frequency is 10 MHz, so minimum target frequency is twice that; increase baud rate accordingly:
    scope.clock.clkgen_freq = 20e6
    target.baud = 104000
    scope.clock.adc_src = "clkgen_x4"
    scope.adc.samples = 40000
    scope.gain.setGain(20)

else:
    %run "Helper_Scripts/Setup_CW305_DST.ipynb"
    scope.adc.samples = 35000
    trace = TraceWhisperer(target, scope)

In [ ]:
# target info and buildtimes:
print(trace.get_target_name())
print(trace.get_fw_buildtime())
print(trace.get_fpga_buildtime())

### Trigger trace capture from target FW:

In [ ]:
trace.use_soft_trigger()

### What to capture:
There are two trace capture modes:
1. Raw mode captures the raw trace data.
2. Non-raw mode captures only matching rule IDs. To use this, set up some pattern match rules (see below); only the ID of the matching rule will be captured.

In [ ]:
trace.fpga_write(trace.REG_CAPTURE_RAW, [0])

### Alternatively, set a pattern matching rule and capture only rule match IDs:

In [ ]:
# match on any PC match (isync) trace packet:
trace.set_pattern_match(0, [3, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])

# enable matching rule:
trace.fpga_write(trace.REG_PATTERN_ENABLE, [1])

### Optionally, a pattern matching rule can be used to trigger trace capture (instead of the target FW soft trigger):

Be aware that this is not a stable trigger, expect jitter up to 6 clock cycles.

In [ ]:
trace.use_trace_trigger(rule=0)

### How long to capture for:
We capture for `REG_CAPTURE_LEN` counts.

What's counted? If `REG_COUNT_WRITES` is set, then it is capture FIFO writes (trace bytes and timestamps) that are counted.

Otherwise, it is clock cycles that are counted.

In [ ]:
trace.fpga_write(trace.REG_COUNT_WRITES, [0])
trace.fpga_write(trace.REG_CAPTURE_LEN, int.to_bytes(40000, length=4, byteorder='little'))

### Set PC addresses to match on:
Let's use the start of the `SubBytes()` and `MixColumns()` functions:

In [ ]:
if TRACE_PLATFORM == 'CW610':
    trace.set_isync_matches(addr0=0x1d60, addr1=0x1d68, match='both')
else:
    trace.set_isync_matches(addr0=0x3bc0, addr1=0x3aa8, match='both')

### Enable or disable periodic PC sampling:

In [ ]:
trace.set_periodic_pc_sampling(enable=0)

# Capture power and debug trace:

In [ ]:
if TRACE_PLATFORM == 'CW610':
    print("*** Don't forget the jumper cable from CW308 GPIO4/TRIG pin to PhyWhisperer PC pin on side connector! ***")

In [ ]:
# force resynchronization, ensure we are sync'd:
trace.resync()

In [ ]:
# arm trace sniffer:
trace.arm_trace()

In [ ]:
if TRACE_PLATFORM == 'CW610':
    sstarget = target
else:
    sstarget = trace._ss

In [ ]:
from tqdm import tnrange

ktp = cw.ktp.Basic()

powertraces = []
num_traces = 1

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    powertrace = cw.capture_trace(scope, sstarget, text, key)
    if powertrace is None:
        continue
    powertraces.append(powertrace)

### Read the raw trace data:

In [ ]:
raw = trace.read_capture_data()
len(raw)

### If we captured raw data, parse out raw 'frames' from it (using sync frames as delimiters):

In [ ]:
frames = trace.get_raw_trace_packets(raw, removesyncs=True, verbose=True)

In [ ]:
len(frames)

#### Each entry of `frames` contains a timestamp (# of clock cycles elapsed since trigger) and a payload:

In [ ]:
frames

### If we captured matching rule events, this will list timestamped matching rule IDs:

In [ ]:
times = trace.get_rule_match_times(raw, rawtimes=False, verbose=True)

In [ ]:
len(times)

# Parse the raw trace data with Orbuculum:
(For the case where REG_CAPTURE_RAW = 1 only.)

In [ ]:
# first, write out the raw trace data to a file:
trace.write_raw_capture(frames, 'raw.bin')

In [ ]:
# change the path to where the orbuculum executable resides on your own system:

In [ ]:
%%bash
/home/jp/github/orbuculum/ofiles/orbuculum -t -f raw.bin -P -e
cat hwevent

Refer to Orbuculum documentation for more information, but for the example shown here you'll get two types of entries out of Orbuculum:
1. Starts with '2': periodic PC sample; last field is the PC value
2. Starts with '8': Isync match; last field is the PC value

The middle field is the timestamp inferred by Orbuculum, which is inaccurate here since TraceWhisperer strips out most of the sync frames for storage efficiency and records its own timestamps instead.

# Plotting Example
For the code below, go back above and re-run a trace capture with non-raw capture mode, using one or two PC addresses that are of interest to you.
Skip over the Orbuculum cells since we aren't capturing raw trace packets.

The default PC match values, for the target executable in the repository, are the start of the `SubBytes()` and `MixColumns()` functions.

The code below overlays black vertical lines on top of the power trace, for each rule match event.

Note that 18 matches are obtained (not 20) because the last round uses a different code path.

In [ ]:
if scope.clock.adc_src == 'clkgen_x4' or scope.clock.adc_src == 'extclk_x4':
    multiplier = 4
else:
    multiplier = 1

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
p = figure(plot_width=1200)

xrange = range(len(powertraces[0].wave))
p.line(xrange, powertraces[0].wave, line_color="red")

vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=2))
p.renderers.extend(vlines)

In [ ]:
show(p)